# スワップ獲り太郎

In [1]:
import datetime
from math import floor
import pprint
import schedule
from statistics import mean
from statistics import median
from statistics import StatisticsError
import sys
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['font.family'] = 'IPAPGothic'
from oandapyV20 import API
from oandapyV20.exceptions import V20Error
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.pricing as pricing
from oandapyV20.endpoints.pricing import PricingInfo
from oandapyV20.contrib.requests import (MarketOrderRequest, StopLossDetails)
import oandapyV20.endpoints.forexlabs as labs
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import selenium.common.exceptions as selex

sys.path.append('../')
from oandapyV21 import *

# OANDA API v20の口座IDとAPIトークン(房林蛇ver)
accountID = "101-009-13234797-001"
access_token = "9739ebcc49088322e07e334ad3697b04-16314025c95d08a3d40882b027316211"

In [2]:
def swap_long(units, inst, print_=False):
    api = API(access_token=access_token, environment="practice")
    
    params = {"instruments": "USD_JPY"}
    pi = PricingInfo(accountID=accountID, params=params)
    api.request(pi)
    
    if pi.response['prices'][0]['status'] == 'tradeable':
        # position check
        p = positions.OpenPositions(accountID=accountID)
        api.request(p)
        if p.response['positions'] == []:
            data = {
              "order": {
                "instrument": inst,
                "units": "+{}".format(units),
                "type": "MARKET",
                "positionFill": "DEFAULT"
              }
            }
            oc = orders.OrderCreate(accountID, data=data)
            api.request(oc)
            if print_:
                pprint.pprint(oc.response)

In [3]:
def swap_short(units, inst):
    time.sleep(5)
    api = API(access_token=access_token, environment="practice")
    
    params = {"instruments": "USD_JPY"}
    pi = PricingInfo(accountID=accountID, params=params)
    api.request(pi)
    
    if pi.response['prices'][0]['status'] == 'tradeable':
        fee = pip_spread(inst)[2]
        # 指値注文がない場合のみ
        ol = orders.OrderList(accountID=accountID)
        api.request(ol)
        if ol.response['orders'] == []:
            # ポジション、指値価格取得
            pd = positions.PositionDetails(accountID=accountID, instrument=inst)
            api.request(pd)
            p = round(float(pd.response['position']['long']['averagePrice']) + fee, 3)
            # 現在価格取得
            params = {"instruments": inst}
            pi = PricingInfo(accountID=accountID, params=params)
            api.request(pi)
            cp = float(pi.response['prices'][0]['bids'][0]['price'])
            # 現在価格が指値価格を上回っていたらそのまま利確
            if cp > p:
                position_data = {"longUnits": "ALL"}
                pc = positions.PositionClose( accountID=accountID, data=position_data, instrument=inst )
                api.request(pc)
            # そうでなければ損益が+-0になる価格に指値
            else:
                data = {
                    'order': {
                        'units': '-{}'.format(units),
                        'instrument': inst,
                        'price': str(p),
                        'type': 'LIMIT',
                        'positionFill': 'DEFAULT'
                    }
                }
                oc = orders.OrderCreate(accountID=accountID, data=data)
                api.request(oc)

In [4]:
# スケジュール作成
units = 250000
inst = 'USD_JPY'
schedule.every().day.at('06:30').do(swap_long, units, inst)
schedule.every().day.at('07:00').do(swap_short, units, inst)

Every 1 day at 07:00:00 do swap_short(250000, 'USD_JPY') (last run: [never], next run: 2020-02-17 07:00:00)

In [5]:
# スケジュール実行
while True:
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 

In [30]:
# スケジュールクリア
schedule.clear()